参考项目：examples/NavigatorAI

In [6]:
import requests

url = "http://localhost:5001/extract_travel_info"
data = {"query": "我想深圳玩3天，帮我规划一下行程"}
response = requests.post(url, json=data)
print(response.json())



{'city': '深圳', 'days': 3, 'need_more_info': False, 'query': '我想深圳玩3天，帮我规划一下行程', 'response': '信息在Navigator的数据库中查询到啦，正在努力为您生成攻略~'}


In [11]:
import requests
import json
import time
import os

def test_travel_planner_api(city: str, days: int):
    """
    用于测试TravelPlanner Flask API的客户端函数。

    Args:
        city (str): 想要规划的城市名称.
        days (int): 想要规划的天数.
    """
    # 1. 配置请求的目标URL和数据
    #    请确保这个地址和端口与您运行的Flask服务完全一致
    api_url = "http://127.0.0.1:5002/get_travel_plan"
    
    # 准备要发送的JSON数据体 (Payload)
    request_payload = {
        "city": city,
        "days": days
    }
    
    print("="*60)
    print(f"🚀 开始测试API: 为【{city}】规划【{days}】天的行程...")
    print(f"请求目标 URL: {api_url}")
    print(f"发送的JSON数据: {json.dumps(request_payload, ensure_ascii=False)}")
    print("⏳ 后端正在处理，AI生成内容可能需要1-2分钟，请耐心等待...")
    print("="*60)
    
    start_time = time.time()
    
    try:
        # 2. 发送 POST 请求
        #    - `json=request_payload` 会自动将字典转为JSON并设置正确的Content-Type请求头
        #    - `timeout=300` 设置了5分钟的超时时间，以防AI处理过慢
        response = requests.post(url=api_url, json=request_payload, timeout=300)
        
        # 3. 检查HTTP响应状态码
        #    如果状态码是 4xx 或 5xx (客户端/服务器错误), 这行代码会自动抛出异常
        response.raise_for_status()
        
        end_time = time.time()
        
        print(f"\n✅ 请求成功！ (HTTP Status Code: {response.status_code})")
        print(f"⏱️ 后端处理总耗时: {end_time - start_time:.2f} 秒")
        
        # 4. 解析并保存后端返回的JSON数据
        #    我们称之为 response_content 或 response_data，而不是 html_content
        response_data = response.json()
        
        # 定义客户端保存响应的文件名
        output_filename = f"client_received_response_{city}_{days}天.json"
        
        with open(output_filename, 'w', encoding='utf-8') as f:
            # 使用 json.dump 将解析后的Python字典美观地写入文件
            # indent=4 让JSON格式化，带缩进，更易读
            # ensure_ascii=False 确保中文字符能被正确写入，而不是被转义
            json.dump(response_data, f, ensure_ascii=False, indent=4)
            
        print(f"💾 后端返回的JSON响应已完整保存到文件: '{os.path.abspath(output_filename)}'")
        
        # 5. 在控制台打印一部分返回的数据作为预览
        print("\n--- 后端返回数据预览 ---")
        if response_data.get('status') == 'success':
            preview_data = response_data.get('data', {})
            print(f"  城市: {preview_data.get('city')}")
            print(f"  天数: {preview_data.get('days')}")
            print(f"  景点数量: {len(preview_data.get('景点', []))}")
            print(f"  美食数量: {len(preview_data.get('美食', []))}")
            print(f"  (更多详情请查看 .json 文件)")
        else:
            # 如果后端返回了错误状态，打印整个响应
            print(json.dumps(response_data, ensure_ascii=False, indent=2))
        print("------------------------\n")

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ HTTP错误发生: {http_err}")
        print(f"   响应内容: {http_err.response.text}")
    except requests.exceptions.ConnectionError:
        print("❌ 连接错误: 无法连接到服务器。")
        print("   请确认您的Flask应用(后端)是否已在另一个终端中成功运行在 http://127.0.0.1:5002 上。")
    except requests.exceptions.Timeout:
        print("❌ 请求超时: 后端处理时间超过了设定的5分钟，可能服务负载过高或出现问题。")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")

if __name__ == '__main__':
    # --- 您可以在这里修改想要测试的城市和天数 ---
    test_city = "深圳"
    test_days = 3
    
    test_travel_planner_api(city=test_city, days=test_days)

🚀 开始测试API: 为【深圳】规划【3】天的行程...
请求目标 URL: http://127.0.0.1:5002/get_travel_plan
发送的JSON数据: {"city": "深圳", "days": 3}
⏳ 后端正在处理，AI生成内容可能需要1-2分钟，请耐心等待...

✅ 请求成功！ (HTTP Status Code: 200)
⏱️ 后端处理总耗时: 174.32 秒
💾 后端返回的JSON响应已完整保存到文件: 'd:\handy-multi-agent-main\code\第五章\client_received_response_深圳_3天.json'

--- 后端返回数据预览 ---
  城市: 深圳
  天数: 3
  景点数量: 12
  美食数量: 19
  (更多详情请查看 .json 文件)
------------------------



In [14]:
import requests
import json
import os

# API端点（与后端端口一致）
url = "http://localhost:5004/generate_itinerary_html"

# 请求数据（确保与JSON文件名匹配，例如"成都3天旅游信息.json"）
data = {
    "city": "深圳",
    "days": "3"
}

try:
    # 发送POST请求
    response = requests.post(url, json=data)

    # 检查响应状态
    if response.status_code == 200:
        result = response.json()
        html_content = result.get("html_content")
        file_path = result.get("file_path")  # 后端返回的保存路径

        if html_content:
            print(f"后端已将HTML保存至：{file_path}")
            
            # （可选）如果需要在测试代码中额外保存，可沿用以下逻辑
            # storage_dir = "第五章"
            # os.makedirs(storage_dir, exist_ok=True)
            # city = data.get("city", "")
            # days = data.get("days", "")
            # filename = f"{storage_dir}/{city}{days}天旅游攻略.html"
            # with open(filename, "w", encoding="utf-8") as f:
            #     f.write(html_content)
            # print(f"测试代码额外保存HTML至：{filename}")
        else:
            print("未获取到有效的html_content")
    else:
        print(f"请求失败: {response.status_code}")
        print(f"错误信息: {response.text}")

except requests.exceptions.RequestException as e:
    print(f"发送请求时发生错误: {e}")













后端已将HTML保存至：d:\handy-multi-agent-main\code\第五章\第五章/深圳3天旅游攻略.html
